In [48]:
#regex for special characters
from random import random
from tweet_collector.modules.tweet_collector import *
import os, re, pandas as pd
from datetime import datetime
from definitions import ROOT_DIR
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

In [153]:
politifact = pd.read_csv(ROOT_DIR+"/data/fakenews_sources/politifact_scape_2609.csv")

dont_sample = politifact[politifact.claim.str.contains('don\'t')]

sample = dont_sample.sample(10, random_state=42).reset_index()

test = 0

sample.claim[test]

'The United States has "10,000 IRS agents making sure that you don\'t take an improper charity deduction," but to fight terrorism, it has "less than two dozen people focusing on countering violent extremism at home."'

In [152]:
def remove_excluded_words(text):
    stop_words = set(stopwords.words('english'))
    
    tweet_tokenizer = TweetTokenizer()
    word_tokens = tweet_tokenizer.tokenize(text)
    #stopword removal
    word_tokens = [w for w in word_tokens if not w.lower() in stop_words]
    #special characters
    word_tokens = [w for w in word_tokens if not re.match(r"[^A-Za-z0-9]", w)]
    filtered_sentence = []
    
    return " ".join(word_tokens)


In [155]:

remove_excluded_words(sample.claim[test])

'United States 10,000 IRS agents making sure take improper charity deduction fight terrorism less two dozen people focusing countering violent extremism home'

In [156]:
def set_character_limit(character_limit, text):
    tweet_tokenizer = TweetTokenizer()
    word_tokens = tweet_tokenizer.tokenize(text) 
    
    sentence = [" ".join([w]) for w in word_tokens if len(sentence)<character_limit]
    
    return sentence
    

In [138]:

def clean_claims(sample):
    # #Cleaning step 1 - getting rid of \n appearances and weird spaces
    # sample['claim'] = sample.claim.apply(lambda x: x.strip())

    # #Cleaning from all special characters - note the \w'\w is to avoid removing the ' in don't, won't they've etc.
    # sample['claim'] = sample.claim.apply(lambda x: re.sub(r'[^a-zA-Z0-9 \w\'\w \.,]', '', x))

    # Removing "Says" in the beginning of a statement around 1000 claim quotes in the sample have this sentence structure.
    # Fine to remove as the quote after is the essence of the claim.
    sample['claim'] = sample.claim.apply(lambda x: re.sub(r'^Says', '', x).strip())
    
    # remove stopwords - done in order to avoid query errors such as:
    # "There were errors processing your request: Ambiguous use of and as a keyword. Use a space to logically join two clauses, or \"and\" to find occurrences of and in text
    sample['claim'] = sample.claim.apply(lambda x: remove_excluded_words(x))
    

def reshape_date_format(sample):
    sample['date'] = sample.date.apply(lambda x: str(datetime.strptime(x, '%B %d, %Y').isoformat()+"Z"))
    

In [139]:
    
#Cleaning and preparation
clean_claims(sample)
reshape_date_format(sample)



In [150]:
#sample.claim[test][0:140]

sample.claim[0]

'United States 10,000 IRS agents making sure take improper charity deduction fight terrorism less two dozen people focusing countering violent extremism home'

In [70]:
date = 'November 25, 2018'
datet = str(datetime.strptime(date, '%B %d, %Y').isoformat()+"Z")

In [71]:
datet

'2018-11-25T00:00:00Z'

In [53]:
date_str = '09-19-2018'

date_object = datetime.strptime(date_str, '%m-%d-%Y')[0:-4].date()


<class 'datetime.date'>


In [74]:
str(datetime.today().isoformat()[0:-4]+"Z")

'2022-09-29T11:19:05.86Z'